# Interprétabilité des modèles
## 0. Overview

Les enjeux derrière l'interprétabilité des modèles, ses motivations ainsi que des méthodes pour les appliquer ont été présentées par **Maxence Brochard**. Vous pouvez à tout moment retrouver les ressources ici : https://lion.app.box.com/folder/65170147483 

=> **L'objectif de ce notebook est la mise en pratique de l'interprétabilité des modèles sur des cas concrets**

### 0.1 Données

Pour la suite de ce notebook, nous nous baserons sur 2 jeux de données connus :
* **boston** : ce dataset vise à déterminer le prix de vente des maisons de Boston en fonction de différents indicateurs
* **iris** : ce dataset vise à classifier des iris suivant 3 catégories en fonction de différentes informations sur les fleurs

Nous commencerons par utiliser le jeu de données **boston**

In [ ]:
from sklearn.datasets import load_boston
import pandas as pd

boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns=["Houses prices"])
print(boston.DESCR)

## 1. Focus sur les Random Forests en Régression

Pour commencer, vous devez installer le package **treeinterpreter** : `pip install treeinterpreter` dans votre CLI.

### 1.1 Import des packages

In [ ]:
import warnings # On enlève les warnings pour la suite du notebook
warnings.filterwarnings('ignore')

In [ ]:
from treeinterpreter import treeinterpreter as ti, utils
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.grid_search import GridSearchCV

### 1.2 Idée générale

Nous allons essayer de nous intéresser un peu plus en profondeur aux Random Forests, et plus particulièrement aux prédictions obtenus par ces modèles, en cherchant à les décomposer.

#### 1.2.1 Première décomposition - Simplification du problème

Dans un premier temps, nous essaierons de décomposer le prix estimé pour une maison *i* comme la somme des contributions de chaque variable pour cette maison, i.e. : 
<center> $ prediction^i=biais+contributionVariable_1^i+…+contributionVariable_n^i $ </center> 

Peu de packages proposent actuellement de rentrer dans ce niveau de détails à l'heure actuelle. Nous nous baserons sur **treeinterpreter**. Ce dernier propose la décomposition exposée au dessus pour différents modèles existants sous *scikit-learn*, tels que :
* DecisionTreeRegressor
* DecisionTreeClassifier
* ExtraTreeRegressor
* ExtraTreeClassifier
* RandomForestRegressor
* RandomForestClassifier
* ExtraTreesRegressor
* ExtraTreesClassifier


#### 1.2.2 Première Random Forest
**Exercice 1 :**
Pour commencer, séparez les données à disposition en 2 échantillons (via la fonction *train_test_split*) en définissant une graîne aléatoire (=1234) :
* un échantillon d'apprentissage (2/3 des données)
* un échantillon de validation (1/3 des données)

Entraînez ensuite une Random Forest (les paramètres par défaut suffiront pour l'exemple) sur la base de vos données d'apprentissage, en indiquant une graine aléatoire (=1234) pour figer les résultats.

*NB : Nous noterons X_train, X_test, y_train, y_test nos échantillons d'apprentissage et de validation, et rf le modèle entraîné*

Nous allons maintenant choisir 2 points de données de notre échantillon de test, sur lequel nous allons prédire notre cible.

In [ ]:
tworows = X_test.iloc[:2,]
for i,prediction in enumerate(rf.predict(tworows)):
    print("Prédiction pour ligne {} de notre échantillon de test : {}".format(i,round(prediction,2)))

On constate que les prédictions sont très éloignées pour ces 2 points de données. L'idée est donc de comprendre maintenant quelles sont les variables qui ont le plus contribuées (aussi bien négativement que positivement) aux prédictions.

Pour cela, nous allons utiliser le package **treeinterpreter**.

La structure est relativement simple, et nous permet, sur la base d'un modèle déjà entraîné, de récupérer pour des points de données de l'échantillon de test, la prédiction du modèle, le biais, ainsi que les contributions de chaque variable :

In [ ]:
prediction, biais, contributions = ti.predict(rf, tworows)

Si l'on commence par s'intéresser au contenu de *prediction*, on remarque bien qu'on récupère les mêmes valeurs que ci-dessus. 

In [ ]:
prediction

Le contenu de la variable *biais*, qui correspond à la moyenne sur l'échantillon d'apprentissage, est logiquement le même qu'importe le point de données de test.

Nous reviendrons dessus par la suite.

In [ ]:
biais

Enfin, la variable *contributions* contient deux arrays de dimensions *1x13* chacun, représentant pour chaque prédiction, la contribution de chacune des variables.

In [ ]:
contributions

On a donc tout ce qui est nécessaire pour déterminer les contributions de chacune des variables aux deux prédictions, en rappelant que :
<center> $ prediction=biais+contributionVariable1+…+contributionVariablen $ </center> 

In [ ]:
for i in range(len(tworows)):
    print("Point de données {}".format(i))
    print("Biais {}".format(biais[i]))
    print("Contributions des variables (par décroissance absolue) :")
    for c, feature in sorted(zip(contributions[i], 
                                 boston.feature_names), 
                             key=lambda x: -abs(x[0])):
        print("{} : {}".format(feature, round(c, 2)))
    print("-"*20) 

**Exercice 2** : A partir des informations récupérées (*biais* et *contributions*) grâce au module *.predict* du package **treeinterpreter**, recalculer les prédictions pour nos 2 points de données de test, et vérifier qu'elles correspondent bien à ce que nous avions obtenu avec *scikit-learn*.

Ce package est donc très pratique pour pouvoir mieux interpréter les prédictions de nos modèles de Random Forests pour certains points de données.

On y trouve **2 intérêts majeurs** :
* Comprendre pourquoi les valeurs prédites sur 2 jeux de données sont différentes, et quelles sont les variables en causes. Sur nos données d'exemple, on pourrait par exemple chercher à comprendre d'où viennent les différences de prix des maisons de plusieurs voisinages
* Débugger un modèle et/ou les données, en cherchant par exemple à comprendre pourquoi les valeurs prédites sur un nouveau jeu de données ne matchent pas avec celle d'anciennes données

=> **Essayons de développer le premier cas**.

**Exercice 3** : Splitter le jeu de données de test *(X_test)* en 2 sous échantillons (respectivement *ech1* & *ech2*) de tailles égales. En utilisant le Random Forest déjà entraîné, calculez et stockez les prédictions associées à ces 2 sous échantillons. Calculez en ensuite la moyenne.

On peut constater que les prédictions moyennes sont relativement différentes sur les 2 échantillons.

**Exerice 4** : Appliquer la décomposition vue au dessus sur les 2 sous échantillons *ech1* et *ech2*. On notera *prediction1, biais1* et *contributions1* (respectivement *2*) les variables dans lequelles seront stockées les résultats.

Moyennez ensuite les contributions par variable pour chaque sous échantillon et stockez les résultats dans deux variables, respectivement *totalc1* et *totalc2*.

Dans la mesure où les biais sont les mêmes (puisque calculés sur le même échantillon d'apprentissage), la différence entre les prédictions moyennes sur les 2 sous échantillons provient uniquement des contributions des différentes variables.
En outre, la différence entre les contributions des variables sommée est égale à la différence entre les prédictions moyennes. Ce que nous pouvons facilement vérifier.


In [ ]:
np.sum(totalc1 - totalc2),np.mean(prediction1) - np.mean(prediction2)

**Exercice 5** : Calculer les différences de contributions de chaque variable entre les 2 sous échantillons, et appuyez vous sur le dictionnaire des données disponible en début de notebook pour interpréter.

#### 1.2.3 Seconde décomposition

Rendre les prévisions de nos Random Forests intérprétable semble donc assez simple, et relativement proche de ce que l'on connaît avec des modèles linéaires.

Cependant, **cette décomposition est imparfaite, puisqu'elle ne prend pas en compte les intéractions entre variables.**
Pour l'illustrer, prenons l'exemple du XOR (= ou exclusif).

##### 1.2.3.1 L'exemple du XOR

In [ ]:
pd.DataFrame({'X1': [0, 0, 1, 1], 'X2': [0, 1, 0, 1], 'Sortie': [0, 1, 1, 0]})

Le **XOR** est relativement facile à comprendre. Pour que la valeur de sortie soit Vraie (=1), il faut que l'une seule des valeurs de *X1* ou *X2* soit Vraie (=1).

Dans ce cas, ni *X1* ni *X2* n'apporte de l'information sur la valeur de sortie seul. C'est à dire que si l'on isole *X1* ou *X2*, il n'est pas possible de prédire la valeur de *Sortie*.

Un arbre de décision va être capable d'apprendre de cet effet et donc de classifier correctement le XOR (arbre ci dessous).

<img src="Tree.png" alt="Drawing" style="width: 400px;" align="left"/>

A nouveau, si l'on se penche sur le premier noeud de l'arbre, où seul *X1* est connu, nous ne sommes pas en mesure de savoir si la valeur de *Sortie* est 1 ou 0. La meilleure prédiction possible est donc 0.5 (ce qui revient à dire "Je ne sais pas"). De ce fait, la contribution de *X1* si l'on ne considère que ce noeud serait (à tord) 0. 

Au noeud suivant de l'arbre, nous prenons connaissance de la valeur de *X2*. Nous pouvons donc aisément réaliser la bonne prédiction.
Toutefois, attribuer la bonne prédiction à la variable *X2* seule serait faux, puisque **c'est l'intéraction des deux variables qui nous permet de bien prédire la valeur de *sortie*.**
Il faut donc répartir la contribution équitablement entre les deux variables.

Prenons le <font color='red'>chemin rouge</font> à titre d'exemple. Ce chemin conduit à la prédiction de la valeur 0.

On sait que la prédiction moyenne du modèle est de 0.50 (=2/4). Pour prédire la valeur 0.00, la contribution de *X1* (i.e. premier noeud) est de 0.00. Et la contribution de l'intéraction *X1X2* est de -0.50.

On a donc bien :
<center> $ 0.00 = 0.50 (contrib X1) - 0.5 (contrib X1X2) $ </center> 


Les **effets d'intéractions** peuvent être calculés par le package **treeinterpreter** en passant le paramètre *joint_contribution=True* à la fonction *.predict*.

**Exercice 6**:  Repartir des 2 sous échantillons *ech1* et *ech2* construit à l'**Exercice 3**. Appliquer de nouveau la décomposition vue à l'exercice 4 en faisant attention à inclure les effets d'intéractions. Stockez à nouveau les résultats dans les variables *prediction1, bias1, contributions1* (respectivement *2* pour l'*ech2*)



La fonction aggregated_contribution du module utils (utils.aggregated_contribution), avec en paramètre les contributions obtenues par la fonction .predict est très utile lorsque l'on utilise des effets d'intéractions.


In [ ]:
mean_contrib1 = utils.aggregated_contribution(contributions1)
mean_contrib2 = utils.aggregated_contribution(contributions2)
mean_contrib1

**Exercice 7** : Observez le contenu de la variable *mean_contrib1*. Quel est son type ? A quoi correspond son contenu ?

<span style="text-decoration:underline">Réponse</span> : 

**Exercice 8** : Vérifiez que la différence entre les contributions des variables (et intéractions de variables) sommée est égale à la différence entre les prédictions moyennes.

**Exercice 9** : En sachant que pour un indice de variable donné, il est possible d'en récupérer le nom via le code suivant :
`indice_variable = 1
boston["feature_names"][indice_variable]`


Calculer de nouveau les différences de contributions entre les 2 sous échantillons, en prenant cette fois en compte les intéractions entre variables.
Afficher par exemple, les 10 variables/intéractions de variables qui ont le plus contribuées aux écarts de prix entre les 2 échantillons.

# Focus sur les Random Forests en Classification

Nous utiliserons ici le dataset **iris**.

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.DataFrame(iris.target, columns=["Classe"])
print(iris.DESCR)

**Exercice Bilan** : Sur la base des données *iris*, appliquez une Random Forest et utiliser le package treeinterpreter pour décomposer la prédiction pour certain point de données choisit manuellement.

Les fonctions vues précédemment s'utilisent de la même manière mais les sorties sont différentes ;)

# Conclusion

A finir

# Crédits
 http://blog.datadive.net/random-forest-interpretation-with-scikit-learn/